# Trabajo Práctico 3 - Lenguaje

# Generación de texto con N-gramas

## Limpieza de corpus de textos en español

In [2]:
import re

# Función para cargar el archivo de texto
def load_text(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        text = file.read()
    return text

# Función para limpiar el texto
def clean_text(text):
    # Eliminar contenido entre corchetes
    text = re.sub(r'\[.*?\]', '', text)
    
    # Convertir texto a minúsculas
    text = text.lower()
    
    # Eliminar puntuación, excepto puntos finales que delimitan oraciones
    text = re.sub(r'[^\w\s\.]', '', text)
    
    # Opcional: eliminar números si no son relevantes
    text = re.sub(r'\d+', '', text)
    
    # Eliminar espacios adicionales
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

# Cargar el texto en español
filename = 'opensubtitles_es.txt'
raw_text = load_text(filename)

# Limpiar el texto
cleaned_text = clean_text(raw_text)

# Guardar el texto limpio
with open('opensubtitles_es_limpio.txt', 'w', encoding='utf-8') as file:
    file.write(cleaned_text)

# Cargar el texto en ingles
filename = 'opensubtitles_en.txt'
raw_text = load_text(filename)

# Limpiar el texto
cleaned_text = clean_text(raw_text)

# Guardar el texto limpio
with open('opensubtitles_en_limpio.txt', 'w', encoding='utf-8') as file:
    file.write(cleaned_text)

## Modelo de N-gramas

**Paso 1: Cargar el Archivo de Texto Limpio**

In [3]:
# Cargar el texto limpio en una variable
with open('opensubtitles_es_limpio.txt', 'r', encoding='utf-8') as file:
    text_es = file.read()

with open('opensubtitles_en_limpio.txt', 'r', encoding='utf-8') as file:
    text_en = file.read()

**Paso 2: Tokenización del Texto**

In [4]:
import re

# Incluir signos de puntuación en la tokenización
tokens_es = re.findall(r'\b\w+\b|[.!?]', text_es.lower())
tokens_en = re.findall(r'\b\w+\b|[.!?]', text_en.lower())
print("Tokens con puntuación:", tokens_es[:10])
print("Tokens con puntuación:", tokens_en[:10])

Tokens con puntuación: ['sr', '.', 'bond', 'sr', '.', 'bond', 'qué', 'bueno', 'que', 'lo']
Tokens con puntuación: ['fall', 'by', 'scott', 'miller', 'taste', 'for', 'life', 'by', 'scott', 'miller']


**Paso 3: Construcción del Modelo de N-Gramas**

In [ ]:
from collections import defaultdict, Counter

n = 3  # Cambia a trigramas

# Crear el modelo de trigramas
model_es = defaultdict(Counter)
model_en = defaultdict(Counter)

# en español
for i in range(len(tokens_es) - n + 1):
    gram = tuple(tokens_es[i:i + n - 1])  # Dos primeras palabras como clave
    next_word = tokens_es[i + n - 1]      # La tercera palabra es el objetivo
    model_es[gram][next_word] += 1  # Incrementamos la cuenta para el siguiente token

# en inglés
for i in range(len(tokens_en) - n + 1):
    gram = tuple(tokens_en[i:i + n - 1])  # Dos primeras palabras como clave
    next_word = tokens_en[i + n - 1]      # La tercera palabra es el objetivo
    model_en[gram][next_word] += 1  # Incrementamos la cuenta para el siguiente token

**Paso 4: Generación de Texto Basado en el Modelo**

In [15]:
import random

def generate_text_with_end(model, starting_words, max_words=50):
    result = list(starting_words)
    for _ in range(max_words):
        state = tuple(result[-2:])  # Últimas dos palabras
        if state in model and model[state]:
            next_word = random.choices(list(model[state].keys()), weights=model[state].values())[0]
            result.append(next_word)
            # Finalizar si la palabra generada es un signo de puntuación
            if next_word in ['.', '!', '?']:
                break
        else:
            break
    return ' '.join(result)

# Probar la generación con finalización automática (español)
starting_words_es = ("la", "casa")
generated_text_es = generate_text_with_end(model_es, starting_words_es, 50)
print("Texto generado:", generated_text_es)

# Probar la generación con finalización automática (inglés)
starting_words_en = ("how", "are")
generated_text_en = generate_text_with_end(model_en, starting_words_en, 50)
print("Texto generado:", generated_text_en)

Texto generado: la casa se incendió .
Texto generado: how are you curious about this is the fundamental resource that gives me fiirst call over to the bishop gone your ass over here sol yeah under control fucker s head especially when he finish him off me i il be in three two .


## Análisis de los Textos Generados

**Diversidad léxica**

In [7]:
def lexical_diversity(text):
    words = text.split()
    unique_words = set(words)
    return len(unique_words) / len(words)

def generate_text_fixed_length(model, starting_words, num_words=50):
    result = list(starting_words)
    for _ in range(num_words - len(starting_words)):  # Ajustamos para alcanzar exactamente `num_words`
        state = tuple(result[-2:])  # Últimas dos palabras
        if state in model and model[state]:
            # Elegir la siguiente palabra basada en las probabilidades del modelo
            next_word = random.choices(list(model[state].keys()), weights=model[state].values())[0]
            result.append(next_word)
        else:
            # Si no hay más opciones, reiniciar con un nuevo estado aleatorio
            state = random.choice(list(model.keys()))
            result.extend(state)
    return ' '.join(result[:num_words])  # Limitar la salida exactamente a `num_words`

# Dividir el texto en fragmentos para impresión
def print_in_chunks(text, chunk_size=20):
    words = text.split()
    for i in range(0, len(words), chunk_size):
        print(' '.join(words[i:i + chunk_size]))



# Generar texto de muestra y calcular la diversidad léxica (español)
starting_words_es = ("es", "muy")
sample_text_es = generate_text_fixed_length(model_es, starting_words_es, 100)  # Genera exactamente 100 palabras
diversity_es = lexical_diversity(sample_text_es)

print("Texto generado en español:")
print_in_chunks(sample_text_es)
print("\nDiversidad léxica:", diversity_es, "\n")

# ---------------------------------------------------------------------------------------------------------

# Generar texto de muestra y calcular la diversidad léxica (inglés)
starting_words_en = ("it", "is")
sample_text_en = generate_text_fixed_length(model_en, starting_words_en, 100)  # Genera exactamente 100 palabras
diversity_en = lexical_diversity(sample_text_en)

print("Texto generado en ingles:")
print_in_chunks(sample_text_en)
print("\nDiversidad léxica:", diversity_en)

Texto generado en español:
es muy peligroso . será para ti . te dije que esperes aquí . times square esquina de la foto
. estoy segura que no pasaremos quién es . quién quién ya sabes alimentar la sopa de pollo . sólo
sabíamos que era marica . no . bien podemos hacerlo con las cuerdas . es el único lugar en este
apartamento . qué tipo de promesa . pero la lleva a la concurrencia . qué astuto eres mírate . has
conocido a una hamburguesa . . . . . y tienes problemas . . . dos millones de dólares .

Diversidad léxica: 0.64 

Texto generado en ingles:
it is . truth is the resume down right embarrassed to find out where the money . me . would
it will test positive for pregnancy miss what are we prisoners or guards please gentlemen . welcome to my friend
simone . you need to play for . . . but the legend of a dangerous species . good morning
joe . at that compass . god this is why i moved in a second here and make something up
johnny . he s home where the car you see toad stool softener i m 

**Longitud promedio de las oraciones**

In [8]:
def average_sentence_length(text):
    sentences = text.split('.')
    return sum(len(sentence.split()) for sentence in sentences) / len(sentences)

# Calcular la longitud promedio de las oraciones (español)
avg_sentence_length_es = average_sentence_length(sample_text_es)
print("Longitud promedio de las oraciones en español:", avg_sentence_length_es)

# Calcular la longitud promedio de las oraciones (inglés)
avg_sentence_length_en = average_sentence_length(sample_text_en)
print("Longitud promedio de las oraciones en ingles:", avg_sentence_length_en)

Longitud promedio de las oraciones en español: 3.391304347826087
Longitud promedio de las oraciones en ingles: 6.769230769230769


**Ratio de Repetición**

In [9]:
from collections import Counter

def repetition_ratio(text):
    words = text.split()
    word_counts = Counter(words)
    repeated_words = sum(1 for count in word_counts.values() if count > 1)
    return repeated_words / len(words)

# Ratio de repetición para el texto en español
repetition_es = repetition_ratio(sample_text_es)
print("Ratio de repetición en español:", repetition_es)

# Ratio de repetición para el texto en inglés
repetition_en = repetition_ratio(sample_text_en)
print("Ratio de repetición en inglés:", repetition_en)

Ratio de repetición en español: 0.09
Ratio de repetición en inglés: 0.1


**Cantidad de Palabras Únicas**

In [10]:
def unique_words_count(text):
    words = text.split()
    unique_words = set(words)
    return len(unique_words)

# Cantidad de palabras únicas en español e inglés
unique_words_es = unique_words_count(sample_text_es)
print("Cantidad de palabras únicas en español:", unique_words_es)

unique_words_en = unique_words_count(sample_text_en)
print("Cantidad de palabras únicas en inglés:", unique_words_en)

Cantidad de palabras únicas en español: 64
Cantidad de palabras únicas en inglés: 74


# Generación de Texto Con GPT-2

In [18]:
pip install transformers torch --quiet

Note: you may need to restart the kernel to use updated packages.


In [28]:
pip install transformers adapter-transformers --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install peft transformers --quiet

Note: you may need to restart the kernel to use updated packages.


**Preprocesamiento y tokenización del corpus**

In [17]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from peft import PeftModel, LoraConfig
from torch.utils.data import Dataset
import wandb

# Cargar el tokenizador y el modelo GPT-2
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Configuración del adaptador Lora
peft_config = LoraConfig(
    r=2,                      # Parámetro reducido para menos parámetros entrenables
    lora_alpha=8,             # Ajuste de Lora alpha
    lora_dropout=0.1,         # Mantenemos dropout para estabilidad
    task_type="CAUSAL_LM"     # Tipo de tarea
)

# Aplicar la configuración al modelo con PEFT
model = PeftModel(model, peft_config)

# Agregar el token de padding
tokenizer.pad_token = tokenizer.eos_token

# Leer el corpus y dividirlo en trozos más pequeños para acelerar el entrenamiento inicial
with open("opensubtitles_en_limpio.txt", "r", encoding="utf-8") as f:
    text = f.read()

# Solo tomar una parte del dataset para acelerar el entrenamiento inicial
subset_size = 100000  # Tamaño reducido de datos para entrenamiento rápido
text = text[:subset_size]

# Tokenizar en fragmentos para evitar problemas de memoria
chunk_size = 512
tokens = []

for i in range(0, len(text), chunk_size):
    chunk = text[i:i + chunk_size]
    tokenized_chunk = tokenizer(chunk, return_tensors="pt", max_length=512, truncation=True, padding="max_length")
    tokens.append(tokenized_chunk["input_ids"])

# Concatenar los fragmentos
input_ids = torch.cat(tokens, dim=0)


/Users/achula/miniconda3/envs/cell_detection/lib/python3.8/site-packages/peft/tuners/lora/layer.py:1150: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


**Entrenamiento de GPT-2 con el corpus de texto**

In [18]:
# Define el dataset
class TextDataset(Dataset):
    def __init__(self, tokens):
        self.tokens = tokens

    def __len__(self):
        return len(self.tokens)

    def __getitem__(self, idx):
        return {
            "input_ids": self.tokens[idx],
            "attention_mask": torch.ones_like(self.tokens[idx]),
            "labels": self.tokens[idx]
        }

train_dataset = TextDataset(input_ids)

# Configuración de argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./gpt2-peft-results",
    overwrite_output_dir=True,
    num_train_epochs=1,                 # Reducido para un entrenamiento rápido inicial
    per_device_train_batch_size=4,      # Incrementado para mejorar la eficiencia
    save_steps=500,
    save_total_limit=2,
    prediction_loss_only=True,
    learning_rate=5e-5,                 # Tasa de aprendizaje incrementada
    report_to="none"                    # Desactivar WandB temporalmente
)

# Definir el entrenador
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)

# Iniciar el entrenamiento
trainer.train()

  0%|          | 0/49 [00:00<?, ?it/s]

{'train_runtime': 45.2974, 'train_samples_per_second': 4.327, 'train_steps_per_second': 1.082, 'train_loss': 10.126649194834183, 'epoch': 1.0}


TrainOutput(global_step=49, training_loss=10.126649194834183, metrics={'train_runtime': 45.2974, 'train_samples_per_second': 4.327, 'train_steps_per_second': 1.082, 'total_flos': 51257630785536.0, 'train_loss': 10.126649194834183, 'epoch': 1.0})

In [1]:
import tensorflow as tf
print(tf.__version__)

2.13.0


In [29]:
# Configuración del modelo y tokenizador en modo evaluación
model.eval()

# Función para generar texto
def generate_text(prompt, min_length=40, max_length=60):  # Rango de longitud
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,          # Longitud máxima permitida
            min_length=min_length,          # Longitud mínima para completar la idea
            eos_token_id=tokenizer.eos_token_id,  
            no_repeat_ngram_size=2,         # Evita repetición de frases
            top_k=30,                       # Controla la variedad de tokens seleccionables
            top_p=0.8,                      # Limita la probabilidad acumulada
            temperature=0.6,                # Reduce la creatividad
            do_sample=True,
            early_stopping=True             # Permite detenerse en un punto natural
        )
        
    # Decodificar el texto generado
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Asegurar que termine con un punto
    if not generated_text.endswith('.'):
        generated_text = generated_text.rstrip() + '.'
        
    return generated_text

# Texto de inicio
prompt_text = "The future of artificial intelligence is"

import textwrap

# Generación de texto
generated_text = generate_text(prompt_text)
print("Texto generado:\n", textwrap.fill(generated_text, width=70))

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Texto generado:
 The future of artificial intelligence is uncertain.  In the coming
years, a new generation of AI will emerge from the world of machine
learning. It will be able to learn from human knowledge and make
decisions about its future. This new AI could also be used to help
people solve complex problems. The.
